In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# To call openai models
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# To use LangSmith
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

#### Convert question to SQL query

In [3]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"'

In [4]:
db.run(response)

'[(8,)]'

In [5]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#### Execute SQL query

In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

#### Answer the question

In [7]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are a total of 8 employees.'

### Agents
##### LangChain has an SQL Agent which provides a more flexible way of interacting with SQL databases.

In [8]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

In [9]:
agent_executor.invoke(
    {
        "input": "How many employees are there"
    }
)

{'input': 'How many employees are there',
 'output': 'There are a total of 8 employees in the database.'}

In [31]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country are as follows:\n1. USA: $523.06\n2. Canada: $303.96\n3. France: $195.10\n4. Brazil: $190.10\n5. Germany: $156.48\n\nThe country whose customers spent the most is the USA with a total sales amount of $523.06.'}

In [32]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})

{'input': 'Describe the playlisttrack table',
 'output': "The `PlaylistTrack` table has the following columns:\n- PlaylistId (INTEGER, NOT NULL)\n- TrackId (INTEGER, NOT NULL)\n\nIt has a composite primary key on the columns PlaylistId and TrackId. Additionally, there are foreign key constraints on TrackId referencing the Track table's TrackId column, and on PlaylistId referencing the Playlist table's PlaylistId column.\n\nHere are 3 sample rows from the `PlaylistTrack` table:\n1. PlaylistId: 1, TrackId: 3402\n2. PlaylistId: 1, TrackId: 3389\n3. PlaylistId: 1, TrackId: 3390"}

In [33]:
agent_executor.invoke({"input": "create RDF model for table customer"})

{'input': 'create RDF model for table customer',
 'output': 'I have retrieved the schema for the "Customer" table. Here is the RDF model for the "Customer" table:\n\n```turtle\n@prefix ex: <http://example.com/> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\nex:Customer rdf:type ex:Table ;\n    ex:hasColumn ex:CustomerId ;\n    ex:hasColumn ex:FirstName ;\n    ex:hasColumn ex:LastName ;\n    ex:hasColumn ex:Company ;\n    ex:hasColumn ex:Address ;\n    ex:hasColumn ex:City ;\n    ex:hasColumn ex:State ;\n    ex:hasColumn ex:Country ;\n    ex:hasColumn ex:PostalCode ;\n    ex:hasColumn ex:Phone ;\n    ex:hasColumn ex:Fax ;\n    ex:hasColumn ex:Email ;\n    ex:hasColumn ex:SupportRepId .\n\nex:CustomerId rdf:type ex:Column ;\n    rdfs:label "CustomerId" .\n\nex:FirstName rdf:type ex:Column ;\n    rdfs:label "FirstName" .\n\nex:LastName rdf:type ex:Column ;\n    rdfs:label "LastName" .\n\nex:Company rdf:type ex:Co

In [34]:
agent_executor.invoke({"input": "Generate sales KPIs"})

{'input': 'Suggest KPIs for Sales',
 'output': 'Based on the database schema, here are some suggested Key Performance Indicators (KPIs) for Sales:\n\n1. Total Sales Revenue: Calculate the total revenue generated from sales by summing the "Total" column in the Invoice table.\n\n```sql\nSELECT SUM(Total) AS TotalSalesRevenue\nFROM Invoice;\n```\n\n2. Number of Invoices: Count the total number of invoices issued.\n\n```sql\nSELECT COUNT(*) AS NumberOfInvoices\nFROM Invoice;\n```\n\n3. Average Order Value: Calculate the average value of each order by dividing the total sales revenue by the number of invoices.\n\n```sql\nSELECT AVG(Total) AS AverageOrderValue\nFROM Invoice;\n```\n\n4. Top Selling Tracks: Identify the top selling tracks based on the quantity sold.\n\n```sql\nSELECT TrackId, SUM(Quantity) AS TotalQuantitySold\nFROM InvoiceLine\nGROUP BY TrackId\nORDER BY TotalQuantitySold DESC\nLIMIT 10;\n```\n\n5. Sales by Country: Analyze sales performance by country.\n\n```sql\nSELECT Bill